## Dados TJPB

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

plt.rcParams['figure.figsize'] = (12,10)
sns.set_theme()

# 1 Preparação dos dados

## 1.1 Leitura dos dados

In [87]:
procs = pd.read_csv('PA_2021124355_.csv',
                    encoding='utf-8-sig')
# procs = procs.sort_values(by='Data da movimentação')
# renomeia colunas
procs.columns = ['desc', 'timestamp',
                 'id_processo', 'classe', 'assunto', 'org_julg',
                 'comp', 'comarca', 'valor_causa', 'status']
procs['timestamp'] = pd.to_datetime(procs['timestamp'])
procs.head()


,desc,timestamp,id_processo,classe,assunto,org_julg,comp,comarca,valor_causa,status
0,Distribuído por sorteio,2016-04-01 17:40:40.228,0805763-93.2016.8.15.0001,MONITÓRIA,Cheque,9ª Vara Cível de Campina Grande,Cível (Art. 164 LOJE-PB),Campina Grande - Fórum de Campina Grande,6900.99,Arquivado
1,Distribuído por sorteio,2016-10-03 11:25:24.804,0818406-83.2016.8.15.0001,PROCEDIMENTO DO JUIZADO ESPECIAL CÍVEL,Cláusulas Abusivas,1º Juizado Especial Cível de Campina Grande,Juizado Especial Cível (Art. 201 LOJE-PB),Campina Grande - Fórum de Campina Grande,7669.92,Arquivado
2,Conclusos para decisão,2016-10-03 11:26:11.137,0818406-83.2016.8.15.0001,PROCEDIMENTO DO JUIZADO ESPECIAL CÍVEL,Cláusulas Abusivas,1º Juizado Especial Cível de Campina Grande,Juizado Especial Cível (Art. 201 LOJE-PB),Campina Grande - Fórum de Campina Grande,7669.92,Arquivado
3,Não Concedida a Antecipação de tutela,2016-10-18 17:22:44.098,0818406-83.2016.8.15.0001,PROCEDIMENTO DO JUIZADO ESPECIAL CÍVEL,Cláusulas Abusivas,1º Juizado Especial Cível de Campina Grande,Juizado Especial Cível (Art. 201 LOJE-PB),Campina Grande - Fórum de Campina Grande,7669.92,Arquivado
4,Expedição de Aviso de recebimento (AR).,2016-10-24 11:57:57.924,0818406-83.2016.8.15.0001,PROCEDIMENTO DO JUIZADO ESPECIAL CÍVEL,Cláusulas Abusivas,1º Juizado Especial Cível de Campina Grande,Juizado Especial Cível (Art. 201 LOJE-PB),Campina Grande - Fórum de Campina Grande,7669.92,Arquivado


**Dados iniciais**

In [88]:
procs.nunique()

desc            545864
timestamp      3846998
id_processo     119098
classe             142
assunto            982
org_julg            42
comp                22
comarca              1
valor_causa      41936
status               2
dtype: int64

Temos 119,098 processos, com 142 classes, 982 assuntos e 42 órgãos julgadores diferentes.

## 1.2 Limpeza da base

Primeiro, são removidos os processos oriundos de varas criminais, uma vez que estão fora do escopo proposto para este trabalho.

In [89]:
# lista varas criminais
orgaos_criminais = [orgao for orgao in procs['org_julg'].unique(
) if re.search('crim|entorp|júri|violência', orgao, flags=re.I)]
print(orgaos_criminais)

['Juizado de Violência Doméstica e Familiar contra a Mulher de Campina Grande', '1ª Vara Criminal de Campina Grande', 'Juizado Especial Criminal de Campina Grande', 'Vara de Entorpecentes de Campina Grande', '5ª Vara Criminal de Campina Grande', '2ª Vara do Tribunal do Júri de Campina Grande', '2ª Vara Criminal de Campina Grande', '3ª Vara Criminal de Campina Grande', '4ª Vara Criminal de Campina Grande', '1ª Vara do Tribunal do Júri de Campina Grande']


In [90]:
# remove processos criminais
procs = procs[~procs['org_julg'].isin(orgaos_criminais)]

In [91]:
procs.nunique()

desc            380508
timestamp      3338518
id_processo     100722
classe             112
assunto            771
org_julg            32
comp                20
comarca              1
valor_causa      41930
status               2
dtype: int64

Abaixo, é ajustada a coluna de descrição das movimentações, substituindo caracteres estranhos por inconsistências no encoding.

In [68]:
# Limpeza da descrição — Inconsistências de enconding
procs['desc'] = (
    procs['desc']
    .str.replace('Ãª', 'ê', regex=False)
    .str.replace('Ã§', 'ç', regex=False)
)


In [71]:
procs.nunique()

desc            380502
timestamp      3338518
id_processo     100722
classe             112
assunto            771
org_julg            32
comp                20
comarca              1
valor_causa      41930
status               2
dtype: int64

## 1.2 Tranformação da base
Num primeiro momento, a base de movimentações será transformada de modo a calcular a duração de cada processo.
Examinaremos a coluna *desc* (descrição da movimentação) para verificar qual movimentação marca o fim do processo, havendo este sido finalizado

In [1]:
# extrai número caracterizador da movimentação, entre colchetes
# Exemplo: "[941]"

procs['num_desc'] = procs['desc'].str.extract('\[(\d+)\]')
procs.head()

NameError: name 'procs' is not defined

In [ ]:
procs.groupby('id_processo')['']

In [73]:
# Lê arquivo de movimentações que caracterizam o julgamento
# Fonte: DITEC - TJPB
movs_julgamento = pd.read_excel('mov_julgamentos.xlsx')
movs_julgamento['Cód'] = movs_julgamento['Cód'].astype(str)
movs_julgamento.head()

,Cód,Movimento,Caminho completo
0,385,Absolvido sumariamente o réu - art. 397 do CPP,Magistrado (1) | Julgamento (193) | Com Resolu...
1,10964,Extinto o Processo por Cumprimento da Medida S...,Magistrado (1) | Julgamento (193) | Com Resolu...
2,11401,Julgado procedente o pedido e procedente o ped...,Magistrado (1) | Julgamento (193) | Com Resolu...
3,11402,Julgado procedente o pedido e procedente em pa...,Magistrado (1) | Julgamento (193) | Com Resolu...
4,11403,Julgado procedente o pedido e improcedente o p...,Magistrado (1) | Julgamento (193) | Com Resolu...


In [76]:
# procs.loc[procs['desc'].isin(movs_julgamento['Movimento'])].shape
procs.loc[procs['desc'].str.rstrip('.') .isin(movs_julgamento['Movimento'])].shape

(78933, 11)

In [ ]:
procs[procs['num_desc'].isin(movs_julgamento['Cód'])]

In [ ]:
procs.loc[procs['desc'].str.contains('Extinto',flags=re.I),'desc']

In [ ]:
procs.head()

In [ ]:
procs['timestamp'].describe(datetime_is_numeric=True)

In [ ]:
procs.loc[procs['desc'].str.contains('\d'), 'id_processo'].nunique()

In [ ]:
procs.drop_duplicates(subset='id_processo') ['status'].value_counts()

In [ ]:
procs['id_processo'].nunique()

In [ ]:
procs['assunto'].isna().sum()

In [ ]:
# procs.groupby('id_processo')['classe'].nunique().value_counts()
procs.groupby('id_processo')['assunto'].nunique().value_counts()

In [ ]:
procs.groupby('id_processo')['desc'].first()#.value_counts().head(20)

In [ ]:
procs['desc'].value_counts().head(25)

In [ ]:
list(range(-10,0))

In [ ]:
# procs.query("status=='Arquivado'") .groupby('id_processo')['desc'].nth([-1,-2, -3]).value_counts().head(25)
procs.query("status=='Arquivado'") .groupby('id_processo')['desc'].nth([-1,-2, -3]).value_counts().head(25)


In [ ]:
# procs.loc[procs['desc'].str.contains('extint|senten', flags=re.I), 'desc'].value_counts().head(25)
procs.loc[procs['desc'].str.contains('decis[aã]o', flags=re.I), 'desc'].value_counts().head(25)

In [ ]:
slice(-5,0)

In [ ]:
procs.groupby('id_processo')['desc'].nth(1).value_counts()

In [ ]:
procs.groupby('Nº do Processo')['Descrição da movimentação'].last().value_counts().head(10)

In [ ]:
procs['Status'].value_counts()